In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sns
import cartopy.crs as ccrs
import pandas as pd
import tarfile

from matplotlib.ticker import FormatStrFormatter
from matplotlib import gridspec, colorbar, colors

C:\Users\ylinh\anaconda3\envs\nasa\lib\site-packages\pyproj\__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [10]:
path = 'C:/Users/ylinh/OneDrive - UW/NASA Sea Ice/data/'

ceres_synoptic = xr.open_dataset(path+'CERES_SYN1deg-Month_Terra-Aqua-MODIS_Ed4.1_Subset_200003-202203 (1).nc')
# Load weighted and deseasonalized CERES data from weighted_avg.ipynb
df = pd.read_pickle('weighted_deseasonalized_avg_CERES')

# Open and extract NSIDC seaice area data
icearea = tarfile.open(path+'icearea.tar', mode='r')
icearea.extractall()

In [11]:
ceres = xr.open_dataset(path+'CERES_EBAF_Ed4.1_Subset_200003-202203.nc') # Arctic all-time data
ocn_fraction = np.loadtxt(path+'water_one_degree.dat')
ocn_fraction = np.reshape(ocn_fraction, (180, 360))

lat = np.array(ceres['lat'])
lon = np.array(ceres['lon'])
lonGrid, latGrid = np.meshgrid(lon, lat)

ocn_fraction = np.flip(ocn_fraction,axis=0)
ocn_fraction = ocn_fraction[-30:,:]
print(ocn_fraction.shape)

# Use lat/lon coords from subset CERES Arctic data
lat = np.array(ceres['lat'])
lon = np.array(ceres['lon'])
lonGrid, latGrid = np.meshgrid(lon, lat)

(30, 360)


In [12]:
filenames = icearea.getnames()
filenames.remove(filenames[0])
filenames.sort()
filenames

seaice_area = []
dates = pd.date_range(start='1/01/2000', end='12/15/2017', freq='MS')
for i, file in enumerate(filenames):
    seaice_area = np.append(seaice_area, np.loadtxt(path+file, usecols=1))
#     print('Reading '+str(file))
seaice_area = pd.DataFrame(seaice_area, columns=['seaice_area'], index=dates) 

In [13]:
## Average 12 variables above 60N weighted zonally * ocn fraction
var_list = ['toa_sw_all_mon', 
'toa_lw_all_mon', 
'toa_net_all_mon', 
'solar_mon', 
'cldarea_total_daynight_mon', 
'cldpress_total_daynight_mon', 
'cldtau_total_day_mon', 
'cldtemp_total_daynight_mon', 
'sfc_sw_down_all_mon', 
'sfc_sw_up_all_mon', 
'sfc_lw_down_all_mon', 
'sfc_lw_up_all_mon']

weight = np.cos(np.deg2rad(latGrid))*ocn_fraction
var_mean = np.zeros((12, 265))

for i, var in enumerate(var_list):
    var_mean[i] = np.array([np.average(ceres[var][j], weights=weight) for j in range(len(ceres[var]))])
#     print('Average found for ' + var + ' at index ' + str(i))
ceres_df = pd.DataFrame(var_mean.transpose(), columns=var_list, index=ceres['time'].to_numpy()) # Pre-deseasonlized EBAF df

In [14]:
# Defining and grouping CERES Variables
ebaf_var = [
    'toa_sw_all_mon', 
'toa_lw_all_mon', 
'toa_net_all_mon', 
'solar_mon', 
'cldarea_total_daynight_mon', 
'cldpress_total_daynight_mon', 
'cldtau_total_day_mon', 
'cldtemp_total_daynight_mon', 
'sfc_sw_down_all_mon', 
'sfc_sw_up_all_mon', 
'sfc_lw_down_all_mon', 
'sfc_lw_up_all_mon']

# ebaf_TOA = ebaf_var[0:4]
# ebaf_sfc = ebaf_var[8:12]
# ebaf_cloud = ebaf_var[4:8]

synoptic_var = [
    'toa_sw_all_mon',
'toa_lw_all_mon',
'cldarea_total_mon',
'cldtau_total_mon',
'lwp_total_mon',
'iwp_total_mon',
'cldwatrad_total_mon',
'cldicerad_total_mon',
'cldphase_total_mon',
'ini_sfc_sw_up_all_mon',
'ini_sfc_sw_down_all_mon',
'ini_albedo_mon']

# synoptic_TOA = synoptic_var[0:2]
# synoptic_sfc = synoptic_var[9:12]
# synoptic_cloud = synoptic_var[2:9]

# Weighting and deseasonalizing CERES Synoptic data
weight = np.cos(np.deg2rad(latGrid))*ocn_fraction
var_mean = np.zeros((len(synoptic_var), 265))
months = ceres_synoptic.groupby("time.month").groups

for i, var in enumerate(synoptic_var):
    month_mean = np.zeros((12,23))
    for j, month in enumerate(months):
#         print('Averaging month '+str(month)+' in '+str(var))
        
        ceres_month = ceres_synoptic.isel(time=months[month])
        weighted_mean = np.array([np.average(ceres_month[var][n], weights=weight) for n in range(len(ceres_month[var]))])
        alltime_mean = np.average(weighted_mean)
        deseason = weighted_mean - alltime_mean
        
        if len(deseason)==22:
            deseason = np.append(deseason, [np.inf])
            # I do this for two reasons: 
            # one, because we have 23 years of data for March but 22 years of data for all other months
            # so I use inf here so that the deseason array can be inserted into month_mean array
            # two, this inf value will be deleted and optical depth has nan values so an inf value must be used
        month_mean[j] = deseason
        
    month_mean = month_mean.flatten('C') # needs 'F', don't understand why but look at cloud optical depth when 'C' is called
    month_mean = month_mean[~np.isinf(month_mean)]
    var_mean[i] = month_mean
#     print('Average found for ' + var + ' at index ' + str(i))

synoptic_df = pd.DataFrame(var_mean.transpose(), columns=synoptic_var, index=ceres_synoptic['time'].to_numpy())
synoptic_df = synoptic_df[synoptic_var]

In [15]:
df = df[(df.index.year >= 2000) & (df.index.year <= 2017)]
ceres_df = ceres_df[(ceres_df.index.year >= 2000) & (ceres_df.index.year <= 2017)]
synoptic_df = synoptic_df[(synoptic_df.index.year >= 2000) & (synoptic_df.index.year <= 2017)]

In [16]:
df['sfc_sw_net'] = df['sfc_sw_down_all_mon'] - df['sfc_sw_up_all_mon']
ceres_df['sfc_sw_net'] = ceres_df['sfc_sw_down_all_mon'] - ceres_df['sfc_sw_up_all_mon']

df['sfc_lw_net'] = df['sfc_lw_down_all_mon'] - df['sfc_lw_up_all_mon']
ceres_df['sfc_lw_net'] = ceres_df['sfc_lw_down_all_mon'] - ceres_df['sfc_lw_up_all_mon']

synoptic_df['ini_sfc_sw_net'] = synoptic_df['ini_sfc_sw_down_all_mon'] - synoptic_df['ini_sfc_sw_up_all_mon']

In [ ]:
df['sfc_alb'] = df['sfc_sw_down_all_mon'] / df['sfc_sw_up_all_mon']
ceres_df['sfc_alb'] = ceres_df['sfc_sw_down_all_mon'] / ceres_df['sfc_sw_up_all_mon']

df['emissivity'] = df['sfc_sw_up_all_mon'] / df['solar_mon']
ceres_df['emissivity'] = ceres_df['sfc_sw_up_all_mon'] / ceres_df['solar_mon']

ebaf_var = [
#     'toa_sw_all_mon', 
# 'toa_lw_all_mon', 
# 'toa_net_all_mon', 
# 'solar_mon', 
'cldarea_total_daynight_mon', 
# 'cldpress_total_daynight_mon', 
'cldtau_total_day_mon', 
'cldtemp_total_daynight_mon', 
# 'sfc_sw_down_all_mon', 
# 'sfc_sw_up_all_mon', 
# 'sfc_lw_down_all_mon', 
'sfc_lw_up_all_mon']

# ebaf_TOA = ebaf_var[0:4]
# ebaf_sfc = ebaf_var[8:12]
# ebaf_cloud = ebaf_var[4:8]

synoptic_var = [
#     'toa_sw_all_mon',
'toa_lw_all_mon',
'cldarea_total_mon',
'cldtau_total_mon',
'lwp_total_mon',
'iwp_total_mon',
# 'cldwatrad_total_mon',
# 'cldicerad_total_mon',
# 'cldphase_total_mon',
'ini_sfc_sw_up_all_mon',
# 'ini_sfc_sw_down_all_mon',
'ini_albedo_mon']

df = df[ebaf_var]
ceres_df = ceres_df[ebaf_var]
synoptic_df = synoptic_df[synoptic_var]

In [ ]:
df.corr()[['toa_net_all_mon']].sort_values(by='toa_net_all_mon', ascending=False)
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(df.corr()[['toa_net_all_mon']].sort_values(by='toa_net_all_mon', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with TOA Net Flux', fontdict={'fontsize':18}, pad=16);

In [ ]:
# CERES EBAF TOA fluxes correlation with CERES EBAF cloud properties + Sea Ice Area
corr = df[ebaf_TOA + ebaf_cloud + ['sfc_sw_net']].corr()

fig, ax = plt.subplots(figsize=(12, 5))
# sns.heatmap(corr, vmin=-1, vmax=1, cmap='BrBG', annot=True, ax=ax1)
heatmap = sns.heatmap(corr.loc[ebaf_TOA, ebaf_cloud + ['sfc_sw_net']], vmin=-1, vmax=1, cmap='BrBG', annot=True)
heatmap.set_title('CERES EBAF TOA Fluxes - CERES EBAF Cloud Properties', fontdict={'fontsize':14})

plt.tight_layout()
plt.show()

In [ ]:
# CERES EBAF SURFACE fluxes correlation with CERES EBAF cloud properties + Sea Ice Area
corr = df[ebaf_sfc + ebaf_cloud + ['sfc_sw_net']].corr()

fig, ax = plt.subplots(figsize=(12, 5))
# sns.heatmap(corr, vmin=-1, vmax=1, cmap='BrBG', annot=True, ax=ax1)
heatmap = sns.heatmap(corr.loc[ebaf_sfc, ebaf_cloud + ['sfc_sw_net']], vmin=-1, vmax=1, cmap='BrBG', annot=True)
heatmap.set_title('CERES EBAF SFC Fluxes - CERES EBAF Cloud Properties', fontdict={'fontsize':14})

plt.tight_layout()
plt.show()

In [ ]:
# CERES Synoptic all variable correlation + Sea Ice Area
plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(synoptic_df.corr(), dtype=np.bool_))
heatmap = sns.heatmap(synoptic_df.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap of CERES Synoptic Data', fontdict={'fontsize':18});

In [ ]:
# CERES synoptic TOA correlation with CERES Synoptic Cloud Properties + Sea Ice Area
corr = synoptic_df[synoptic_TOA + synoptic_cloud].corr()

fig, ax = plt.subplots(figsize=(12, 5))
heatmap = sns.heatmap(corr.loc[synoptic_TOA, synoptic_cloud], vmin=-1, vmax=1, cmap='BrBG', annot=True)
heatmap.set_title('CERES Synoptic TOA - CERES Synoptic Cloud Properties', fontdict={'fontsize':14})

plt.tight_layout()
plt.show()

In [ ]:
df.to_pickle('EBAF-anom')
ceres_df.to_pickle('EBAF')
synoptic_df.to_pickle('synoptic-anom')

# Correlation with sea ice

In [ ]:
plt.plot(seaice_area['seaice_area']/np.max(seaice_area['seaice_area']), label='sea ice')
plt.plot(df['toa_sw_all_mon']/np.max(df['toa_sw_all_mon']), label='shortwave')
plt.legend()

In [ ]:
plt.plot(df['cldarea_total_daynight_mon']/np.max(df['cldarea_total_daynight_mon']), label='cloud area')
plt.plot(df['toa_sw_all_mon']/np.max(df['toa_sw_all_mon']), label='shortwave')
plt.legend()

In [ ]:
def corrWith(df, seaice_area):
    seaice_area = seaice_area.reset_index().drop(columns=['index'])
    
    df = df.reset_index().drop(columns=['index'])
    df = df.merge(seaice_area, left_index=True, right_index=True, suffixes=(None,None))
    
    df_corr = df.corr()
    df_corr = round(df_corr,2)
    return df_corr

In [ ]:
# CERES Synoptic all variable correlation + Sea Ice Area
df_corr = corrWith(df, seaice_area)

plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df_corr, dtype=np.bool_))
heatmap = sns.heatmap(df_corr, mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap of EBAF Anomaly Data', fontdict={'fontsize':18});

In [ ]:
ceres_df

In [ ]:
# CERES Synoptic all variable correlation + Sea Ice Area
# def corrWith(df, seaice_area):
#     seaice_area = seaice_area.reset_index().drop(columns=['index'])
    
#     df = df.reset_index().drop(columns=['index'])
#     df = df.merge(seaice_area, left_index=True, right_index=True, suffixes=(None,None))
    
#     df_corr = df.corr()
#     df_corr = round(df_corr,2)
#     return df_corr, df
# ceres_df_corr, ceres_df = corrWith(ceres_df, seaice_area)

ceres_df_corr = corrWith(ceres_df, seaice_area)

plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(ceres_df_corr, dtype=np.bool_))
heatmap = sns.heatmap(ceres_df_corr, mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap of CERES EBAF Data', fontdict={'fontsize':18});

In [ ]:
# CERES Synoptic all variable correlation + Sea Ice Area
synoptic_df_corr = corrWith(synoptic_df, seaice_area)

plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(synoptic_df_corr, dtype=np.bool_))
heatmap = sns.heatmap(synoptic_df_corr, mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap of CERES Synoptic Data', fontdict={'fontsize':18});

# Subsetting Mar-Oct to exclude winter months

In [ ]:
df = df[(df.index.month >= 3) & (df.index.month <= 10)]
synoptic_df = synoptic_df[(synoptic_df.index.month >= 3) & (synoptic_df.index.month <= 10)]
ceres_df = ceres_df[(ceres_df.index.month >= 3) & (ceres_df.index.month <= 10)]

In [ ]:
df_corr = corrWith(df, seaice_area)

plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df_corr, dtype=np.bool_))
heatmap = sns.heatmap(df_corr, mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap of EBAF Anomaly Data', fontdict={'fontsize':18});

In [ ]:
# CERES Synoptic all variable correlation + Sea Ice Area
synoptic_df_corr = corrWith(synoptic_df, seaice_area)

plt.figure(figsize=(16, 6))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(synoptic_df_corr, dtype=np.bool_))
heatmap = sns.heatmap(synoptic_df_corr, mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap of CERES Synoptic Data', fontdict={'fontsize':18});

In [ ]:
fig, ax = plt.subplots(2,1)
ax.flatten()

ax[0].plot(ceres_df['sfc_lw_net'], 'b', label='LW')
ax[1].plot(df['sfc_lw_net'],'b', label='LW')
ax[0].plot(ceres_df['sfc_sw_net'], 'r', label='SW')
ax[1].plot(df['sfc_sw_net'], 'r', label='SW')
           
# ax[0].legend()
ax[1].legend()

ax[0].set_xlim(ceres_df.index.date[0],ceres_df.index.date[-1])
ax[1].set_xlim(df.index.date[0],df.index.date[-1])

ax[0].set_ylabel('W m-2')
ax[1].set_ylabel('W m-2')

ax[1].set_xlabel('Year')

ax[0].set_title('TOA Net Flux Weighted Average')
ax[1].set_title('TOA Net Flux Anomalies')

fig.tight_layout()